In [1]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
from os.path import join
import torch
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from os.path import join as oj
import torch.utils.data as utils
from torchvision import datasets, transforms
import numpy as np
import os
import sys
pd.set_option('precision', 3)

In [4]:
save_path = "../results_for_export"
trained_model_folder = '../img_models/'
fnames = sorted([oj(trained_model_folder, fname) for fname in os.listdir(trained_model_folder) if 'block' not in fname]) 
# other models were trained badly

results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in (fnames)] 
results = pd.concat(results_list, axis=1).T.infer_objects() 


In [5]:
results['final_acc'] = [max(x) for x in results['accs_test']] #TODO change so it 
results['final_acc_train'] = [x[-1] for x in results['accs_train']]

results['final_cd'] = [x[-1] for x in results['cd']]
results['final_test_loss'] = [min(x) for x in results['losses_test']]
results['final_train_loss'] = [min(x) for x in results['losses_train']]
results['acc_color'] = [0 for x in results['accs_train']]

In [6]:
results = results.dropna()

In [7]:

results.reset_index(drop=True, inplace=True)

In [8]:
results_save = results[['regularizer_rate','final_acc','final_acc_train','final_test_loss', 'acc_color','final_cd' ]].sort_values(by = ['regularizer_rate'])

In [9]:
results_save = results_save[results_save.final_acc_train > 15] # assume that something has gone wrong otherwise

In [10]:
concise_results = results_save.groupby(by = 'regularizer_rate').mean()
concise_results.columns = ['Test accuracy', 'Train accuracy', 'Test loss', 'Color accuracy', 'CD']
with open(oj(save_path, "color_mnist.text"), 'w') as f:
          f.write(concise_results.to_latex())

In [11]:
concise_results_for_save = concise_results[['Test accuracy', 'Color accuracy']]

In [12]:
concise_results_for_save

,Test accuracy,Color accuracy
regularizer_rate,,
0.0,0.468,0
0.1,4.242,0
1.0,2.130,0
10.0,29.922,0


In [13]:
with open(oj(save_path, 'colormnist.tex'), 'w') as f:
          f.write(concise_results_for_save.to_latex(index  = False))

In [14]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_drop = nn.Dropout2d()

        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc1(x))
        
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    def logits(self, x):
    
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc1(x))
        
        x = self.fc2(x)
        return x


In [15]:
def test( model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    return(test_loss, 100.*correct / len(test_loader.dataset))
#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#     test_loss, correct, len(test_loader.dataset),
#     100. * correct / len(test_loader.dataset)))

In [16]:
kwargs = {'num_workers': 1, 'pin_memory': True}
test_x_tensor = torch.Tensor(np.load(oj("../data/ColorMNIST", "test_x.npy")))
test_y_color= torch.Tensor(np.load(oj("../data/ColorMNIST", "test_y_color.npy"))).type(torch.int64)
test_dataset_color = utils.TensorDataset(test_x_tensor,test_y_color) # create your datset

test_loader_color = utils.DataLoader(test_dataset_color,
        batch_size=256, shuffle=True, **kwargs) # create your dataloader



In [17]:
test_net = Net()
test_net = test_net.to(0)



In [33]:
acc_color_list = []
loss_color_list = []
for i in range(len(results)):
        test_net.load_state_dict(results.model_weights[i])
    loss_col, acc_col = test(test_net, 0, test_loader_color, 0)
    acc_color_list.append(acc_col)
    loss_color_list.append(loss_col)
results["acc_color"] =[x for x in acc_color_list]
results["loss_color"] =[x for x in loss_color_list]               
               


In [22]:
results[['regularizer_rate',"acc_color",'loss_color']].sort_values(by = ['regularizer_rate'])

,regularizer_rate,acc_color,loss_color
0,0.0,92.43,0.381
1,0.0,89.12,0.350
16,0.0,85.75,0.430
3,0.0,97.29,0.242
11,0.0,95.61,0.296
18,0.1,84.38,0.504
8,0.1,90.65,0.438
9,0.1,63.52,0.825
4,0.1,72.16,0.675
19,0.1,87.66,0.442
